# Spam Classifier using NaiveBayes.

In [42]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('../MLCourse/emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('../MLCourse/emails/ham', 'ham'))

data.head()

,message,class
../MLCourse/emails/spam\00001.7848dde101aa985090474a91ec93fcf0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",spam
../MLCourse/emails/spam\00002.d94f1b97e48ed3b553b3508d116e6a09,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
../MLCourse/emails/spam\00003.2ee33bc6eacdb11f38d052c44819ba6c,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
../MLCourse/emails/spam\00004.eac8de8d759b7e74154f142194282724,##############################################...,spam
../MLCourse/emails/spam\00005.57696a39d7d84318ce497886896bf90d,I thought you might like these:\n\n1) Slim Dow...,spam


### Using CountVectorizer

In [43]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

MultinomialNB()

### Trying out an example

In [44]:
examples = ['Free viagra now!!!', "Hi tomorrow is the scheduled meetup"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'], dtype='<U4')

### Trying to apply Train/Test to this.

In [45]:
vectorizer = CountVectorizer()
x = vectorizer.fit_transform(data['message'].values)

encoder = LabelEncoder()
y = encoder.fit_transform(data['class'].values)

In [46]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

nb = MultinomialNB()
nb.fit(x_train, y_train)

MultinomialNB()

In [52]:
y_predicted = nb.predict(x_test)
y_predicted_labels = encoder.inverse_transform(y_predicted) 

In [53]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_predicted)

0.955

In [54]:
ham_m=0
spam_m=0
m_num=0
hpos = []
spos =[]
for i in y_predicted_labels:
    if i == 'ham':
        ham_m +=1
        hpos.append(i.index)
    elif i == 'spam':
        spam_m +=1
        spos.append(i.index)
    m_num +=1
    
avg_ham=ham_m/m_num
avg_spam=spam_m/m_num


In [55]:
avg_ham

0.89

In [56]:
avg_spam

0.11